<a href="https://www.kaggle.com/code/luchales/random-forest-score-77?scriptVersionId=115861977" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
#i make imputations for the missing data in the database
train.pivot_table(index=['SibSp','Parch','Sex'],values='Age',aggfunc=['mean','median'])

mean median
                          Age    Age
SibSp Parch Sex                     
0     0     female  30.150000   28.5
            male    32.901316   30.0
      1     female  27.086957   22.0
            male    33.530000   36.5
      2     female  20.705882   22.0
            male    21.536667   17.0
      3     female  24.000000   24.0
      4     female  29.000000   29.0
      5     female  40.000000   40.0
1     0     female  31.806122   29.0
            male    32.311321   30.0
      1     female  29.160000   31.0
            male    25.258621   19.0
      2     female  21.200000   23.0
            male    19.417143   25.0
      3     female  51.000000   51.0
            male    16.000000   16.0
      4     female  45.000000   45.0
            male    52.000000   52.0
      5     female  38.500000   38.5
            male    39.000000   39.0
      6     female  43.000000   43.0
2     0     female  35.500000   35.5
            male    31.181818   28.0
      1     female   6.900000    4.0
            male    12.000000   12.0
      2     female  17.250000   19.5
      3     female  24.000000   24.0
3     0     female  31.500000   31.5
      1     female   5.500000    5.5
            male     2.000000    2.0
      2     female  14.500000   16.0
            male    11.000000   10.0
4     1     male     6.777778    7.0
      2     female   8.333333    7.5
            male     5.333333    4.0
5     2     female  16.000000   16.0
            male     8.750000   10.0

In [8]:
train['Age']=train['Age'].fillna(train.groupby(['SibSp','Parch','Sex'])['Age'].transform('median'))

In [9]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [10]:
train['Embarked']=train['Embarked'].fillna('S')

In [11]:
train['Age']=train['Age'].fillna(30)

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [13]:
test.pivot_table(index=['SibSp','Parch','Sex'],values='Age',aggfunc=['mean','median'])

mean median
                          Age    Age
SibSp Parch Sex                     
0     0     female  29.205882  28.00
            male    30.262963  27.00
      1     female  33.777778  23.00
            male    22.566000  21.00
      2     female  37.375000  36.00
            male    13.266000   8.00
      3     female  36.000000  36.00
1     0     female  35.440000  30.00
            male    38.410714  37.25
      1     female  23.058824  19.00
            male    30.160714  33.50
      2     female  14.348333  12.50
            male    39.500000  39.50
      3     female  48.000000  48.00
            male    61.000000  61.00
      4     female  60.000000  60.00
      5     male    40.000000  40.00
      6     male    40.000000  40.00
2     0     female  45.333333  55.00
            male    21.000000  21.50
      1     female  16.000000  16.00
      2     male    15.500000  15.50
3     0     male    31.000000  31.00
      1     male    17.500000  17.50
      2     female  28.000000  28.00
4     1     male    10.000000  10.00
      2     female  38.000000  38.00
            male     9.000000   9.00
5     2     female  10.000000  10.00
8     2     male    14.500000  14.50

In [14]:
test['Age']=test['Age'].fillna(train.groupby(['SibSp','Parch','Sex'])['Age'].transform('median'))

In [15]:
test['Fare'].mean()

35.627188489208635

In [16]:
test['Fare']=test['Fare'].fillna(36)

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [18]:
#Normally in emergencies women and children are evacuated, 
#considering the context of the industrial revolution it will be assumed that children under 8 years of age have more priority
#Kids
kids=train.loc[train.Age<=8]['Survived']
rate_kids=sum(kids)/len(kids)
rate_kids

0.6206896551724138

In [19]:
sum(train.Survived)

342

In [20]:
#On the Titanic certain classes had priorities
#Class
Fclass=train.loc[train.Pclass==1]['Survived']
rate_Fclass=sum(Fclass)/len(Fclass)

Sclass=train.loc[train.Pclass==2]['Survived']
rate_Sclass=sum(Sclass)/len(Sclass)

Tclass=train.loc[train.Pclass==3]['Survived']
rate_Tclass=sum(Tclass)/len(Tclass)

print(rate_Fclass,rate_Sclass,rate_Tclass)


0.6296296296296297 0.47282608695652173 0.24236252545824846


In [21]:
#We also look at a combination between two variables, that she is a woman and that she is in the first class
Fcfemale=train[(train.Pclass==1)&(train.Sex=='female')]

In [22]:
len(Fcfemale[(Fcfemale.Survived==1)])/len(Fcfemale)

0.9680851063829787

In [23]:
#We create the new variables in the data
train['Ffemale']=np.where((train.Pclass==1)&(train.Sex=='female'),1,0)
test['Ffemale']=np.where((test.Pclass==1)&(test.Sex=='female'),1,0)
train['kids']=np.where((train.Age<=8),1,0)
test['kids']=np.where((test.Age<=8),1,0)

In [24]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ffemale,kids
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,0


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
train=train.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

In [27]:
#'Sex','Pclass','Embarked'
#Here we make these variables dummies

dummies1 = pd.get_dummies(train['Embarked'])

train = pd.concat([train,dummies1],axis=1)

dummies2 = pd.get_dummies(test['Embarked'])

test = pd.concat([test,dummies2],axis=1)

del train['Embarked']
del test['Embarked']

dummies1 = pd.get_dummies(train['Sex'])

train = pd.concat([train,dummies1],axis=1)

dummies2 = pd.get_dummies(test['Sex'])

test = pd.concat([test,dummies2],axis=1)

del train['Sex']
del test['Sex']

dummies1 = pd.get_dummies(train['Pclass'])

train = pd.concat([train,dummies1],axis=1)

dummies2 = pd.get_dummies(test['Pclass'])

test = pd.concat([test,dummies2],axis=1)

del train['Pclass']
del test['Pclass']

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Age       891 non-null    float64
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
 5   Ffemale   891 non-null    int64  
 6   kids      891 non-null    int64  
 7   C         891 non-null    uint8  
 8   Q         891 non-null    uint8  
 9   S         891 non-null    uint8  
 10  female    891 non-null    uint8  
 11  male      891 non-null    uint8  
 12  1         891 non-null    uint8  
 13  2         891 non-null    uint8  
 14  3         891 non-null    uint8  
dtypes: float64(2), int64(5), uint8(8)
memory usage: 55.8 KB


In [29]:
test=test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
x_t=test.values
#x_t.shape
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Age      418 non-null    float64
 1   SibSp    418 non-null    int64  
 2   Parch    418 non-null    int64  
 3   Fare     418 non-null    float64
 4   Ffemale  418 non-null    int64  
 5   kids     418 non-null    int64  
 6   C        418 non-null    uint8  
 7   Q        418 non-null    uint8  
 8   S        418 non-null    uint8  
 9   female   418 non-null    uint8  
 10  male     418 non-null    uint8  
 11  1        418 non-null    uint8  
 12  2        418 non-null    uint8  
 13  3        418 non-null    uint8  
dtypes: float64(2), int64(4), uint8(8)
memory usage: 23.0 KB


In [30]:
y_df=train['Survived']
x_df=train.drop(['Survived'],axis=1)

In [31]:
y=y_df.values
x=x_df.values
print(y.shape,
x.shape)

(891,) (891, 14)


In [32]:
X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.33, 
                                                    random_state=42)

In [33]:
#import xgboost as xgb
#booster = xgb.XGBClassifier(n_jobs=-1)
#booster = booster.fit(X_train,y_train)
#We make a random forest
from sklearn.ensemble import  RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(n_estimators=100,n_jobs=-1)
rf=rf.fit(X_train,y_train)
acc=rf.score(X_train,y_train)
acc

0.9781879194630873

**The score that the model gives us is 97%, but in kaggle it gives us a score of approximately 76%.**

In [34]:
pred=rf.predict(x_t)
pred1=pd.DataFrame(pred)
pred1=pred1.round().astype(int)

test1=pd.read_csv('/kaggle/input/titanic/test.csv')
Pas=test1['PassengerId']
output=pd.concat([Pas,pred1],axis=1)
output=output.rename(columns={0:'Survived'})
output.to_csv('submission.csv', index=False)
#the score that kaggle gave me was 0.76

# **Last submission kaggle gave me a score of 0.76**